In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-1.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-1        4    2    4    0    2    5     1   0   2  ...  1.000000   
1      prop-1        4   12    4    1   19   44    66   2  18  ...  0.000000   
2      prop-1        4   10    4    0   13   48    45   4  13  ...  1.000000   
3      prop-1        4    2    3    0    5    7     1   0   5  ...  1.000000   
4      prop-1        4    8    1    0    4   38     0   3   1  ...  0.500000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96   9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14   7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0   0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10  26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9  51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0        0  0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1        0  0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2        0  0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3        0  0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4        0  0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[18471 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 185]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom   ca  ce  ...       dam  \
15646  prop-1      185    3    1    0    4   11     3    1   3  ...  0.000000   
15647  prop-1      185   13    1   22   62   96    74   22  40  ...  0.750000   
15648  prop-1      185    2    1    0    2    5     1    1   1  ...  0.000000   
15649  prop-1      185    5    1    0  148    5    10  144   4  ...  0.000000   
15650  prop-1      185   18    2    0   53  146   117    1  53  ...  0.625000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ...  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96    9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14    7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0    0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10   26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9   51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
15646    0  0.000000  0.333333   0    0  22.333333       3  1.6667    0  
15647    1  0.000000  0.333333   0    0  38.230769      13  3.3077    0  
15648    0  0.000000  0.750000   0    0  17.000000       6  3.0000    0  
15649    0  0.000000  1.000000   0    0   0.000000       1  1.0000    0  
15650    0  0.448276  0.264706   1    8  64.222222      35  5.1667    1  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[2825 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc       dam  \
17249   10    3    0   14   27    43   1  13    9  0.989899  316  0.181818   
16855    1    1    0    0    5     0   0   0    0  2.000000   15  0.000000   
15735    6    4    0   25   75     9   1  25    2  0.660000  837  0.900000   
16051    5    2    0   10   15     8   2   9    4  0.750000   43  1.000000   
15842    2    3    0   11   18     0   2   9    2  0.500000   78  0.500000   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...       ...   
17242   13    1    0   17   31    64   3  15   10  0.944444  197  0.333333   
15806    4    1    0   13   20     4   0  13    3  1.000000   94  0.000000   
15706    9    3    2   14   32    34   6   9    3  1.020833  165  0.666667   
17354    1    1    0    7    1     0   7   0    1  2.000000    1  0.000000   
18223    2    1    0    2    3     1   0   2    2  2.000000    6  0.000000   

       moa       mfa       cam  ic  cbm         amc  max_cc   avg_cc  
17249    0  0.619048  0.361111   0    0   29.500000       1   0.8000  
16855    0  0.000000  0.000000   0    0   12.000000       0   0.0000  
15735    0  0.947368  0.309524   2    3  136.833333      31  12.6667  
16051    1  0.862069  0.366667   1    1    7.400000       2   1.0000  
15842    0  0.916667  0.750000   0    0   37.000000       3   1.5000  
...    ...       ...       ...  ..  ...         ...     ...      ...  
17242    0  0.000000  0.416667   0    0   13.692308       2   1.1538  
15806    1  0.000000  0.450000   0    0   22.000000       3   1.5000  
15706    1  0.829787  0.266667   1    2   16.666667       4   1.3333  
17354    0  0.000000  1.000000   0    0    0.000000       1   1.0000  
18223    0  0.000000  0.750000   0    0    2.000000       1   0.5000  

[2260 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3   loc       dam  \
16462    1    2    0    9    2     0   8   1    1  2.000000    14  0.000000   
15714   13    2    0   34   94    76   1  34    1  1.041667   652  0.500000   
17297   11    1    0   18   60    29   2  17    3  0.913333   483  0.933333   
16041   11    5    2    6   15     0   5   1   11  0.100000    97  1.000000   
18029    4    3    0   11   22     4   1  10    2  0.833333   163  0.500000   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   ...       ...   
16571    2    3    0    7   13     1   0   7    2  2.000000    43  0.000000   
16649   10    3    0   21   78     9   1  21    6  0.855556  1004  0.700000   
16151    3    1    0   22    3     3  22   0    3  2.000000     3  0.000000   
17798   11    1    0   15   59    33   0  15    6  1.036957   626  0.934783   
17786    3    2    0   22   35     3   0  22    1  0.500000   138  0.000000   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
16462    0  1.000000  1.000000   0    0   3.000000       0  0.0000  
15714    0  0.520000  0.177515   1    6  48.692308      13  2.9231  
17297    3  0.000000  0.500000   0    0  41.545455       7  1.8182  
16041    0  0.888889  0.581818   0    0   7.727273       1  0.2727  
18029    0  0.750000  0.555556   1    1  39.250000      13  4.0000  
...    ...       ...       ...  ..  ...        ...     ...     ...  
16571    0  0.857143  0.666667   0    0  20.500000       3  1.5000  
16649    0  0.692308  0.203704   0    0  98.400000      13  3.3000  
16151    0  0.000000  1.000000   0    0   0.000000       1  1.0000  
17798    2  0.000000  0.480000   0    0  51.727273       5  2.0909  
17786    0  0.500000  0.500000   0    0  44.666667       1  0.6667  

[565 rows x 20 columns]

In [9]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [10]:
y_train

bug
17249    0
16855    0
15735    3
16051    0
15842    0
...    ...
17242    0
15806    0
15706    0
17354    0
18223    0

[2260 rows x 1 columns]

In [11]:
y_test

bug
16462    0
15714    0
17297    0
16041    0
18029    0
...    ...
16571    0
16649    0
16151    0
17798    1
17786    2

[565 rows x 1 columns]

In [12]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [13]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [14]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [15]:
y = y_test['bug'].tolist()
y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 6,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,


In [16]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_propv185/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
23/23 [==============================] - 1s 2ms/step - loss: 602.8937 - mse: 602.8937 - mae: 11.5182 - root_mean_squared_error: 24.5539 - mean_squared_logarithmic_error: 4.2010
Epoch 2/100
23/23 [==============================] - 0s 1ms/step - loss: 466.9585 - mse: 466.9585 - mae: 10.1455 - root_mean_squared_error: 21.6092 - mean_squared_logarithmic_error: 3.8369
Epoch 3/100
23/23 [==============================] - 0s 1ms/step - loss: 379.5763 - mse: 379.5763 - mae: 9.0454 - root_mean_squared_error: 19.4827 - mean_squared_logarithmic_error: 3.5161
Epoch 4/100
23/23 [==============================] - 0s 1ms/step - loss: 313.0534 - mse: 313.0534 - mae: 8.1696 - root_mean_squared_error: 17.6933 - mean_squared_logarithmic_error: 3.2551
Epoch 5/100
23/23 [==============================] - 0s 1ms/step - loss: 263.4280 - mse: 263.4280 - mae: 7.4524 - root_mean_squared_error: 16.2305 - mean_squared_logarithmic_error: 3.0311
Epoch 6/100
23/23 [=============================

23/23 [==============================] - 0s 1ms/step - loss: 0.4330 - mse: 0.4330 - mae: 0.1620 - root_mean_squared_error: 0.6580 - mean_squared_logarithmic_error: 0.0942
Epoch 89/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4330 - mse: 0.4330 - mae: 0.1620 - root_mean_squared_error: 0.6580 - mean_squared_logarithmic_error: 0.0942
Epoch 90/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4329 - mse: 0.4329 - mae: 0.1620 - root_mean_squared_error: 0.6580 - mean_squared_logarithmic_error: 0.0941
Epoch 91/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4329 - mse: 0.4329 - mae: 0.1620 - root_mean_squared_error: 0.6579 - mean_squared_logarithmic_error: 0.0941
Epoch 92/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4329 - mse: 0.4329 - mae: 0.1620 - root_mean_squared_error: 0.6579 - mean_squared_logarithmic_error: 0.0941
Epoch 93/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4328 - mse: 0.432

23/23 [==============================] - 0s 996us/step - loss: 0.4627 - mse: 0.4627 - mae: 0.1666 - root_mean_squared_error: 0.6802 - mean_squared_logarithmic_error: 0.0961
Epoch 75/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4617 - mse: 0.4617 - mae: 0.1665 - root_mean_squared_error: 0.6795 - mean_squared_logarithmic_error: 0.0961
Epoch 76/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4607 - mse: 0.4607 - mae: 0.1664 - root_mean_squared_error: 0.6787 - mean_squared_logarithmic_error: 0.0960
Epoch 77/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4596 - mse: 0.4596 - mae: 0.1662 - root_mean_squared_error: 0.6780 - mean_squared_logarithmic_error: 0.0959
Epoch 78/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4588 - mse: 0.4588 - mae: 0.1661 - root_mean_squared_error: 0.6774 - mean_squared_logarithmic_error: 0.0959
Epoch 79/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4577 - mse: 0.4

23/23 [==============================] - 0s 858us/step - loss: 0.4362 - mse: 0.4362 - mae: 0.1634 - root_mean_squared_error: 0.6605 - mean_squared_logarithmic_error: 0.0949
Epoch 61/100
23/23 [==============================] - 0s 931us/step - loss: 0.4360 - mse: 0.4360 - mae: 0.1633 - root_mean_squared_error: 0.6603 - mean_squared_logarithmic_error: 0.0949
Epoch 62/100
23/23 [==============================] - 0s 891us/step - loss: 0.4358 - mse: 0.4358 - mae: 0.1633 - root_mean_squared_error: 0.6602 - mean_squared_logarithmic_error: 0.0948
Epoch 63/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4357 - mse: 0.4357 - mae: 0.1632 - root_mean_squared_error: 0.6601 - mean_squared_logarithmic_error: 0.0948
Epoch 64/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4355 - mse: 0.4355 - mae: 0.1632 - root_mean_squared_error: 0.6599 - mean_squared_logarithmic_error: 0.0948
Epoch 65/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4354 - mse:

23/23 [==============================] - 0s 986us/step - loss: 5.1019 - mse: 5.1019 - mae: 1.5371 - root_mean_squared_error: 2.2587 - mean_squared_logarithmic_error: 0.8316
Epoch 3/100
23/23 [==============================] - 0s 1ms/step - loss: 3.1213 - mse: 3.1213 - mae: 1.1347 - root_mean_squared_error: 1.7667 - mean_squared_logarithmic_error: 0.5866
Epoch 4/100
23/23 [==============================] - 0s 1ms/step - loss: 2.2822 - mse: 2.2822 - mae: 0.9115 - root_mean_squared_error: 1.5107 - mean_squared_logarithmic_error: 0.4538
Epoch 5/100
23/23 [==============================] - 0s 997us/step - loss: 1.8429 - mse: 1.8429 - mae: 0.7784 - root_mean_squared_error: 1.3576 - mean_squared_logarithmic_error: 0.3756
Epoch 6/100
23/23 [==============================] - 0s 1ms/step - loss: 1.5759 - mse: 1.5759 - mae: 0.6845 - root_mean_squared_error: 1.2554 - mean_squared_logarithmic_error: 0.3229
Epoch 7/100
23/23 [==============================] - 0s 1ms/step - loss: 1.3859 - mse: 1.3859

...

23/23 [==============================] - 0s 925us/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1615 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 90/100
23/23 [==============================] - 0s 972us/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1615 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 91/100
23/23 [==============================] - 0s 960us/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1615 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 92/100
23/23 [==============================] - 0s 985us/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1615 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 93/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1615 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 94/100
23/23 [==============================] - 0s 950us/step - loss: 0.4323 - 

23/23 [==============================] - 0s 934us/step - loss: 0.4324 - mse: 0.4324 - mae: 0.1617 - root_mean_squared_error: 0.6576 - mean_squared_logarithmic_error: 0.0939
Epoch 75/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4324 - mse: 0.4324 - mae: 0.1617 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 76/100
23/23 [==============================] - 0s 988us/step - loss: 0.4324 - mse: 0.4324 - mae: 0.1617 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 77/100
23/23 [==============================] - 0s 977us/step - loss: 0.4324 - mse: 0.4324 - mae: 0.1617 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 78/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4324 - mse: 0.4324 - mae: 0.1617 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 79/100
23/23 [==============================] - 0s 924us/step - loss: 0.4323 - ms

23/23 [==============================] - 0s 967us/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1615 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 60/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1615 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 61/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1615 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 62/100
23/23 [==============================] - 0s 977us/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1615 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 63/100
23/23 [==============================] - 0s 966us/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1615 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 64/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4323 - mse:

Epoch 1/100
23/23 [==============================] - 0s 3ms/step - loss: 1309.0657 - mse: 1309.0657 - mae: 7.3068 - root_mean_squared_error: 36.1810 - mean_squared_logarithmic_error: 1.9053
Epoch 2/100
23/23 [==============================] - 0s 1ms/step - loss: 990.0287 - mse: 990.0287 - mae: 5.2915 - root_mean_squared_error: 31.4647 - mean_squared_logarithmic_error: 1.3795
Epoch 3/100
23/23 [==============================] - 0s 1ms/step - loss: 795.8180 - mse: 795.8179 - mae: 3.9869 - root_mean_squared_error: 28.2102 - mean_squared_logarithmic_error: 1.0212
Epoch 4/100
23/23 [==============================] - 0s 964us/step - loss: 629.3971 - mse: 629.3971 - mae: 3.1283 - root_mean_squared_error: 25.0878 - mean_squared_logarithmic_error: 0.7952
Epoch 5/100
23/23 [==============================] - 0s 1ms/step - loss: 530.2809 - mse: 530.2809 - mae: 2.5592 - root_mean_squared_error: 23.0278 - mean_squared_logarithmic_error: 0.6491
Epoch 6/100
23/23 [==============================] - 0s 

23/23 [==============================] - 0s 982us/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1616 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 89/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1616 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 90/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1616 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 91/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1616 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 92/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4323 - mse: 0.4323 - mae: 0.1616 - root_mean_squared_error: 0.6575 - mean_squared_logarithmic_error: 0.0939
Epoch 93/100
23/23 [==============================] - 0s 1ms/step - loss: 0.4323 - mse: 0.4

In [17]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_propv185/'

In [18]:
total_training_time

[3.5993614196777344,
 3.389803647994995,
 3.355587959289551,
 3.330200433731079,
 3.2980105876922607,
 3.125775098800659,
 2.849799156188965,
 2.7666163444519043,
 2.6753673553466797,
 2.822807788848877,
 2.7560505867004395,
 2.716132879257202,
 2.8252334594726562,
 2.935392379760742,
 3.202179193496704,
 3.0843629837036133,
 3.233001232147217,
 3.1537022590637207,
 3.213313102722168,
 3.090202808380127,
 3.1025636196136475,
 3.1536779403686523,
 3.211756944656372,
 3.3289051055908203,
 3.204442024230957,
 3.123331308364868,
 3.1006011962890625,
 3.1131093502044678,
 3.126919984817505,
 3.2053685188293457]

In [19]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 3.1031192223231


In [20]:
total_training_loss_history

[{'loss': [602.8937377929688,
   466.9584655761719,
   379.5762634277344,
   313.0533752441406,
   263.427978515625,
   222.8186492919922,
   187.85035705566406,
   160.02764892578125,
   134.9813690185547,
   116.2642593383789,
   98.93529510498047,
   84.35457611083984,
   72.90776062011719,
   62.5888557434082,
   54.2763786315918,
   46.78779602050781,
   40.68038558959961,
   35.51362609863281,
   30.74036979675293,
   26.94321632385254,
   23.577789306640625,
   20.76434898376465,
   9.320669174194336,
   3.7242624759674072,
   1.997313380241394,
   1.217868447303772,
   0.9223310947418213,
   0.7395457625389099,
   0.623176097869873,
   0.5571988821029663,
   0.513553261756897,
   0.49047231674194336,
   0.47613686323165894,
   0.46542516350746155,
   0.4570958614349365,
   0.45140835642814636,
   0.44787025451660156,
   0.4445897936820984,
   0.44237807393074036,
   0.440370112657547,
   0.4391198754310608,
   0.43819504976272583,
   0.4373764991760254,
   0.43677660822868347,


In [21]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 1.27883094449838
avg mae of training: 0.19003668079773586


In [22]:
total_scores

[[0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151],
 [0.6074503660202026,
  0.6074503660202026,
  0.22068148851394653,
  0.7793910503387451,
  0.12764345109462738],
 [0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151],
 [0.5929949879646301,
  0.5929949879646301,
  0.2196480631828308,
  0.7700616717338562,
  0.1240013986825943],
 [0.5925974249839783,
  0.5925974249839783,
  0.2145688533782959,
  0.769803524017334,
  0.12291421741247177],
 [0.6317073702812195,
  0.6317073702812195,
  0.2240234613418579,
  0.7948002219200134,
  0.12985636293888092],
 [0.5659492611885071,
  0.5659492611885071,
  0.22217698395252228,
  0.752295970916748,
  0.12000513821840286],
 [0.7336903214454651,
  0.7336903214454651,
  0.23347283899784088,
  0.8565572500228882,
  0.13454629480838776],
 [0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.124632559

In [4]:
a = [[0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151],
 [0.6074503660202026,
  0.6074503660202026,
  0.22068148851394653,
  0.7793910503387451,
  0.12764345109462738],
 [0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151],
 [0.5929949879646301,
  0.5929949879646301,
  0.2196480631828308,
  0.7700616717338562,
  0.1240013986825943],
 [0.5925974249839783,
  0.5925974249839783,
  0.2145688533782959,
  0.769803524017334,
  0.12291421741247177],
 [0.6317073702812195,
  0.6317073702812195,
  0.2240234613418579,
  0.7948002219200134,
  0.12985636293888092],
 [0.5659492611885071,
  0.5659492611885071,
  0.22217698395252228,
  0.752295970916748,
  0.12000513821840286],
 [0.7336903214454651,
  0.7336903214454651,
  0.23347283899784088,
  0.8565572500228882,
  0.13454629480838776],
 [0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151],
 [1.1447641849517822,
  1.1447641849517822,
  0.2501607835292816,
  1.0699365139007568,
  0.14128100872039795],
 [0.8663730621337891,
  0.8663730621337891,
  0.24933519959449768,
  0.9307916164398193,
  0.14399810135364532],
 [0.5949299335479736,
  0.5949299335479736,
  0.21658046543598175,
  0.7713170051574707,
  0.12480632215738297],
 [0.5807145237922668,
  0.5807145237922668,
  0.22821636497974396,
  0.7620462775230408,
  0.11929914355278015],
 [0.6137062907218933,
  0.6137062907218933,
  0.2220119833946228,
  0.7833940982818604,
  0.12839031219482422],
 [0.671562910079956,
  0.671562910079956,
  0.2275935858488083,
  0.8194894194602966,
  0.13190379738807678],
 [0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151],
 [66.92670440673828,
  66.92670440673828,
  0.7899512648582458,
  8.18087387084961,
  0.22651511430740356],
 [0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151],
 [0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151],
 [0.5962274074554443,
  0.5962274074554443,
  0.21757864952087402,
  0.772157609462738,
  0.1254000961780548],
 [0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151],
 [0.6488396525382996,
  0.6488396525382996,
  0.21948662400245667,
  0.8055058121681213,
  0.11915121227502823],
 [6.691542148590088,
  6.691542148590088,
  0.7536548972129822,
  2.586801528930664,
  0.3544009327888489],
 [0.595595121383667,
  0.595595121383667,
  0.23808231949806213,
  0.7717480659484863,
  0.13258466124534607],
 [0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151],
 [0.5957448482513428,
  0.5957448482513428,
  0.2186419665813446,
  0.7718451023101807,
  0.12533646821975708],
 [0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151],
 [0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151],
 [0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151],
 [0.5946902632713318,
  0.5946902632713318,
  0.21592921018600464,
  0.7711616158485413,
  0.12463255971670151]]

In [5]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])
sum(test_mae)/10

0.223252010345459

In [3]:
test_mae

[0.21592921018600464,
 0.22068148851394653,
 0.21592921018600464,
 0.2196480631828308,
 0.2145688533782959,
 0.2240234613418579,
 0.22217698395252228,
 0.23347283899784088,
 0.21592921018600464,
 0.2501607835292816,
 0.24933519959449768,
 0.21658046543598175,
 0.22821636497974396,
 0.2220119833946228,
 0.2275935858488083,
 0.21592921018600464,
 0.7899512648582458,
 0.21592921018600464,
 0.21592921018600464,
 0.21757864952087402,
 0.21592921018600464,
 0.21948662400245667,
 0.7536548972129822,
 0.23808231949806213,
 0.21592921018600464,
 0.2186419665813446,
 0.21592921018600464,
 0.21592921018600464,
 0.21592921018600464,
 0.21592921018600464]

In [26]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 3.0462459127108255
avg mae: 0.25856721053520837
avg are: 0.12295181825757026


In [27]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.43233905732631683
median testing mse: 0.5946902632713318


In [28]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.16205035150051117
median testing mae: 0.21811030805110931


In [29]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.12035398185253143


In [30]:
test_ARE

[0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.120000005,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.11964601,
 0.12035398,
 0.12035398,
 0.12035398,
 0.18230088,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.119469024,
 0.1379351,
 0.12070796,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398]

In [6]:
sum([0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.120000005,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.11964601,
 0.12035398,
 0.12035398,
 0.12035398,
 0.18230088,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.119469024,
 0.1379351,
 0.12070796,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398,
 0.12035398][:10])/10

0.12031858249999998